In [20]:
import pickle
import numpy as np
import paddle
import pandas as pd
import paddle.nn as nn

t_file = open('data/train_data.pkl', 'rb')
train_data = pickle.load(t_file)
v_file = open('data/verify_data.pkl', 'rb')
verify_data = pickle.load(v_file)

users_df = pd.read_csv('data/users.csv')
items_df = pd.read_csv('data/items.csv')

TypeError: set_default_dtype only supports [float16, float32, float64] , but received int32

In [6]:
emb_scale = 1024
batch_size = 64
len_users = len(users_df)
len_items = len(items_df)

In [8]:
class Dataset(paddle.io.Dataset):
    def __init__(self, data):
        self.feature = data[:,0:2]
        self.label = data[:,2]
        pass
    def __getitem__(self, idx):
        return self.feature[idx], self.label[idx]
    def __len__(self):
        return len(self.feature)
    pass

train_dataset = paddle.io.DataLoader(Dataset(train_data),
                               batch_size=batch_size,
                               shuffle=True)
verify_dataset = paddle.io.DataLoader(Dataset(verify_data),
                               batch_size=batch_size,
                               shuffle=True)                               

In [28]:
Dataset(train_data).label

array([1., 0., 1., ..., 1., 0., 1.])

In [29]:
Dataset(train_data).feature

array([[  396.,  2154.],
       [ 7678.,  5970.],
       [10126.,  3288.],
       ...,
       [ 6048.,  3360.],
       [10389., 11683.],
       [ 9325.,  4965.]])

In [15]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users, emb_scale)
        self.items_emb = nn.Embedding(len_items, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        user = nn.functional.relu(user)
        user = self.users_fc2(user)
        user = nn.functional.relu(user)
        user = self.users_fc3(user)
        user = nn.functional.relu(user)
        user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        item = nn.functional.relu(item)
        item = self.items_fc2(item)
        item = nn.functional.relu(item)
        item = self.items_fc3(item)
        item = nn.functional.relu(item)
        item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=0)
        x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()

In [25]:
dssm.train()
optim = paddle.optimizer.Adam(parameters=dssm.parameters())
for epoch in range(5):
    for batch_id, data in enumerate(train_dataset):
        x_data = data[0]
        y_data = data[0]
        x_data = paddle.cast(x_data, dtype='int32')
        y_data = paddle.cast(y_data, dtype='int32')

        y_pred = dssm(x_data)
        # print(y_pred)
        print(y_data)
        loss = nn.functional.mse_loss(y_pred, y_data)
        acc = paddle.metric.accuracy(y_pred, y_data)
        loss.backward()

        print('epoch:{}, batch_id:{}, loss:{}, acc:{}'.format(epoch, batch_id, loss, acc))
        optim.step()
        optim.clear_grad()

Tensor(shape=[64, 2], dtype=int32, place=Place(cpu), stop_gradient=True,
       [[1146 , 17031],
        [4950 , 10555],
        [4979 , 5716 ],
        [2431 , 3671 ],
        [7472 , 7562 ],
        [4921 , 3812 ],
        [9802 , 1470 ],
        [343  , 19488],
        [2243 , 20133],
        [2448 , 5251 ],
        [4280 , 19240],
        [1575 , 9388 ],
        [2746 , 17934],
        [5900 , 11502],
        [8494 , 5323 ],
        [1879 , 8074 ],
        [7569 , 5612 ],
        [543  , 22135],
        [6790 , 6787 ],
        [8210 , 10629],
        [10980, 18532],
        [10521, 18770],
        [6454 , 5223 ],
        [539  , 4084 ],
        [5959 , 18073],
        [1562 , 9796 ],
        [10087, 11181],
        [7406 , 8037 ],
        [8942 , 21811],
        [11180, 14558],
        [3318 , 6478 ],
        [6565 , 18131],
        [10693, 13031],
        [12070, 1148 ],
        [9087 , 5070 ],
        [3437 , 18640],
        [5363 , 3683 ],
        [8482 , 6594 ],
        [232  ,

ValueError: (InvalidArgument) Broadcast dimension mismatch. Operands could not be broadcast together with the shape of X = [128] and the shape of Y = [64, 2]. Received [128] in X is not equal to [2] in Y at i:1.
  [Hint: Expected x_dims_array[i] == y_dims_array[i] || x_dims_array[i] <= 1 || y_dims_array[i] <= 1 == true, but received x_dims_array[i] == y_dims_array[i] || x_dims_array[i] <= 1 || y_dims_array[i] <= 1:0 != true:1.] (at C:\home\workspace\Paddle_release\paddle/phi/kernels/funcs/common_shape.h:84)
  [operator < elementwise_sub > error]

In [30]:
data

[Tensor(shape=[64, 2], dtype=float64, place=Place(cpu), stop_gradient=True,
        [[1146. , 17031.],
         [4950. , 10555.],
         [4979. , 5716. ],
         [2431. , 3671. ],
         [7472. , 7562. ],
         [4921. , 3812. ],
         [9802. , 1470. ],
         [343.  , 19488.],
         [2243. , 20133.],
         [2448. , 5251. ],
         [4280. , 19240.],
         [1575. , 9388. ],
         [2746. , 17934.],
         [5900. , 11502.],
         [8494. , 5323. ],
         [1879. , 8074. ],
         [7569. , 5612. ],
         [543.  , 22135.],
         [6790. , 6787. ],
         [8210. , 10629.],
         [10980., 18532.],
         [10521., 18770.],
         [6454. , 5223. ],
         [539.  , 4084. ],
         [5959. , 18073.],
         [1562. , 9796. ],
         [10087., 11181.],
         [7406. , 8037. ],
         [8942. , 21811.],
         [11180., 14558.],
         [3318. , 6478. ],
         [6565. , 18131.],
         [10693., 13031.],
         [12070., 1148. ],
      

In [31]:
y_data

Tensor(shape=[64, 2], dtype=int32, place=Place(cpu), stop_gradient=True,
       [[1146 , 17031],
        [4950 , 10555],
        [4979 , 5716 ],
        [2431 , 3671 ],
        [7472 , 7562 ],
        [4921 , 3812 ],
        [9802 , 1470 ],
        [343  , 19488],
        [2243 , 20133],
        [2448 , 5251 ],
        [4280 , 19240],
        [1575 , 9388 ],
        [2746 , 17934],
        [5900 , 11502],
        [8494 , 5323 ],
        [1879 , 8074 ],
        [7569 , 5612 ],
        [543  , 22135],
        [6790 , 6787 ],
        [8210 , 10629],
        [10980, 18532],
        [10521, 18770],
        [6454 , 5223 ],
        [539  , 4084 ],
        [5959 , 18073],
        [1562 , 9796 ],
        [10087, 11181],
        [7406 , 8037 ],
        [8942 , 21811],
        [11180, 14558],
        [3318 , 6478 ],
        [6565 , 18131],
        [10693, 13031],
        [12070, 1148 ],
        [9087 , 5070 ],
        [3437 , 18640],
        [5363 , 3683 ],
        [8482 , 6594 ],
        [232  ,

In [32]:
x_data

Tensor(shape=[64, 2], dtype=int32, place=Place(cpu), stop_gradient=True,
       [[1146 , 17031],
        [4950 , 10555],
        [4979 , 5716 ],
        [2431 , 3671 ],
        [7472 , 7562 ],
        [4921 , 3812 ],
        [9802 , 1470 ],
        [343  , 19488],
        [2243 , 20133],
        [2448 , 5251 ],
        [4280 , 19240],
        [1575 , 9388 ],
        [2746 , 17934],
        [5900 , 11502],
        [8494 , 5323 ],
        [1879 , 8074 ],
        [7569 , 5612 ],
        [543  , 22135],
        [6790 , 6787 ],
        [8210 , 10629],
        [10980, 18532],
        [10521, 18770],
        [6454 , 5223 ],
        [539  , 4084 ],
        [5959 , 18073],
        [1562 , 9796 ],
        [10087, 11181],
        [7406 , 8037 ],
        [8942 , 21811],
        [11180, 14558],
        [3318 , 6478 ],
        [6565 , 18131],
        [10693, 13031],
        [12070, 1148 ],
        [9087 , 5070 ],
        [3437 , 18640],
        [5363 , 3683 ],
        [8482 , 6594 ],
        [232  ,